<a href="https://colab.research.google.com/github/maclandrol/cours-ia-med/blob/master/03_PyTorch_Fundamentals_Medical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03. Fondamentaux PyTorch

**Enseignant:** Emmanuel Noutahi, PhD

---

**Objectif:** Acquérir les bases PyTorch nécessaires pour utiliser des modèles d'IA médicale.

**Concepts essentiels :**
- Tenseurs et opérations de base
- Chargement de modèles pré-entraînés
- Utilisation simple d'HuggingFace
- Inférence basique

## Installation et Configuration

In [ ]:
# Installation des bibliothèques essentielles
!pip install torch torchvision transformers -q
!pip install pandas numpy matplotlib -q

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel

# Configuration de base
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositif utilisé: {device}")

## 1. Tenseurs de Base

Les tenseurs sont la structure de données fondamentale de PyTorch. Ils permettent de manipuler des données numériques efficacement.

In [ ]:
# Création de tenseurs
print("=== CRÉATION DE TENSEURS ===")

# Différentes façons de créer des tenseurs
tensor_zeros = torch.zeros(3, 4)
tensor_ones = torch.ones(2, 3)
tensor_random = torch.randn(2, 3)
tensor_from_list = torch.tensor([1, 2, 3, 4])

print(f"Tenseur de zéros: {tensor_zeros.shape}")
print(f"Tenseur de uns: {tensor_ones.shape}")
print(f"Tenseur aléatoire: {tensor_random.shape}")
print(f"Tenseur depuis liste: {tensor_from_list.shape}")

# Opérations de base
a = torch.tensor([1.0, 2.0, 3.0])
b = torch.tensor([4.0, 5.0, 6.0])

print(f"\nAddition: {a + b}")
print(f"Multiplication: {a * b}")
print(f"Moyenne: {a.mean()}")
print(f"Somme: {a.sum()}")

In [ ]:
# Exemple avec données médicales simples
print("=== EXEMPLE MÉDICAL ===")

# Données de patients (âge, poids, taille)
patients = torch.tensor([
    [65, 70, 1.75],
    [45, 60, 1.60],
    [50, 80, 1.80],
    [30, 55, 1.65]
], dtype=torch.float32)

print(f"Données patients: {patients.shape}")
print(patients)

# Calculs vectorisés simples
ages = patients[:, 0]
poids = patients[:, 1]
tailles = patients[:, 2]

# IMC pour tous les patients en une fois
imc = poids / (tailles ** 2)
print(f"\nIMC des patients: {imc}")
print(f"IMC moyen: {imc.mean():.1f}")

## 2. Modèles et Réseaux de Neurones

Concept de base d'un réseau de neurones en PyTorch.

In [ ]:
# Réseau de neurones simple
print("=== RÉSEAU DE NEURONES SIMPLE ===")

class SimpleNet(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)
    
    def forward(self, x):
        return self.linear(x)

# Création d'un modèle simple
model = SimpleNet(input_size=3, output_size=1)
print(f"Modèle créé avec {sum(p.numel() for p in model.parameters())} paramètres")

# Test avec nos données patients
with torch.no_grad():
    predictions = model(patients)
    
print(f"\nPrédictions: {predictions.shape}")
print(f"Valeurs: {predictions.flatten()}")

## 3. Images et Données 2D

Manipulation basique d'images avec PyTorch.

In [ ]:
# Images simulées
print("=== MANIPULATION D'IMAGES ===")

# Image simulée (1 canal, 64x64 pixels)
image = torch.randn(1, 64, 64)
print(f"Image: {image.shape} [canaux, hauteur, largeur]")

# Lot d'images (batch)
batch_images = torch.randn(4, 1, 64, 64)
print(f"Lot d'images: {batch_images.shape} [batch, canaux, hauteur, largeur]")

# Opérations sur les images
mean_intensity = batch_images.mean()
max_intensity = batch_images.max()
min_intensity = batch_images.min()

print(f"\nIntensité moyenne: {mean_intensity:.3f}")
print(f"Intensité max: {max_intensity:.3f}")
print(f"Intensité min: {min_intensity:.3f}")

# Visualisation d'une image
plt.figure(figsize=(6, 4))
plt.imshow(image[0], cmap='gray')
plt.title('Image simulée')
plt.axis('off')
plt.show()

## 4. Modèles Pré-entraînés avec HuggingFace

Utilisation simple de modèles existants.

In [ ]:
# Chargement d'un modèle pré-entraîné
print("=== MODÈLES PRÉ-ENTRAÎNÉS ===")

try:
    # Modèle français simple
    model_name = "camembert-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    
    print(f"Modèle chargé: {model_name}")
    print(f"Vocabulaire: {len(tokenizer)} mots")
    
    # Test avec texte médical simple
    text = "Patient avec fièvre et toux"
    
    # Tokenisation
    inputs = tokenizer(text, return_tensors="pt")
    print(f"\nTexte: {text}")
    print(f"Tokens: {inputs['input_ids'].shape}")
    
    # Inférence
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state
    
    print(f"Embeddings: {embeddings.shape}")
    print("Modèle prêt pour utilisation")
    
except Exception as e:
    print(f"Erreur: {e}")
    print("Vérifiez votre connexion internet")

## 5. Inférence Simple

Comment utiliser un modèle pour faire des prédictions.

In [ ]:
# Exemple d'inférence simple
print("=== INFÉRENCE SIMPLE ===")

def simple_prediction(model, data):
    """Fonction simple pour faire une prédiction"""
    model.eval()  # Mode évaluation
    with torch.no_grad():  # Pas de calcul de gradients
        prediction = model(data)
    return prediction

# Test avec notre modèle simple
simple_net = SimpleNet(3, 2)  # 3 entrées, 2 sorties

# Nouveau patient
nouveau_patient = torch.tensor([[55.0, 65.0, 1.70]])
print(f"Nouveau patient: âge={nouveau_patient[0,0]}, poids={nouveau_patient[0,1]}, taille={nouveau_patient[0,2]}")

# Prédiction
resultat = simple_prediction(simple_net, nouveau_patient)
print(f"Résultat du modèle: {resultat}")

# Conversion en probabilités
probabilities = torch.softmax(resultat, dim=1)
print(f"Probabilités: {probabilities}")
print(f"Prédiction finale: classe {torch.argmax(probabilities, dim=1).item()}")

## Résumé

### Concepts PyTorch Essentiels Acquis:

1. **Tenseurs**: Structure de données de base pour manipuler des nombres
2. **Opérations**: Addition, multiplication, moyennes sur des lots de données
3. **Réseaux**: Création simple avec `nn.Module` et `nn.Linear`
4. **Images**: Manipulation de données 2D (hauteur, largeur, canaux)
5. **Modèles pré-entraînés**: Utilisation d'HuggingFace pour charger des modèles existants
6. **Inférence**: Mode `eval()` et `torch.no_grad()` pour les prédictions

### Prêt pour la Suite

Vous maîtrisez maintenant les bases PyTorch nécessaires pour utiliser des modèles d'IA médicale. Le prochain notebook vous montrera comment appliquer ces concepts à la classification de textes médicaux en français.